## 1) Importing the Housing Data and looking at the size of the dataset.
5768 rows 14 columns


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Loading the dataset
dataset = pd.read_csv('Master_File_Housing_Data.csv')

dataset.shape

(5768, 14)

## 2) Visualizing the dataset columns

In [2]:
dataset.head()

,APN,FORMATTED_ADDRESS,COMMUNITY,ABOVEGROUNDAREA,BELOWGROUNDAREA,PARCEL_AREA_SQFT,YEARBUILT,BATHROOMS,BEDROOMS,NUM_STORIES,FIREPLACES,CONSTRUCTIONTYPE,Assessed Value,Sale Price
0,302924440099,2910 California St,NORTHEAST,NaN,NaN,10640,NaN,NaN,NaN,NaN,NaN,NaN,320000,352500
1,402824210259,3041 Holmes Ave #401,CALHOUN-ISLE,NaN,NaN,11425,NaN,NaN,NaN,NaN,NaN,NaN,460000,1359150
2,402824240090,3248 Holmes Ave,CALHOUN-ISLE,NaN,NaN,5450,NaN,NaN,NaN,NaN,NaN,NaN,520500,555350
3,602823110060,3117 42ND Ave S,LONGFELLOW,NaN,NaN,6750,NaN,NaN,NaN,NaN,NaN,NaN,350000,650000
4,802824430075,4433 Washburn Ave S,SOUTHWEST,NaN,NaN,5670,NaN,NaN,NaN,NaN,NaN,NaN,900000,1245000


## 3) Data Transformation (part 1)

Start removing all blanks from the categorical data field 'Constructointype' (the only categorical field that was missing any rows).  We were only missing 47 rows in a dataset of >5k instances, so this is an acceptable reduction is dataset size.

Additionally, we have a bunch of blank values in the below ground square footage column (likely because of no basement), but we need to put in 0 so that we can run our models.

In [3]:
df= dataset.dropna(how='any', subset=['CONSTRUCTIONTYPE'])
df['BELOWGROUNDAREA'].fillna(0,inplace=True)

X = df.iloc[:,2:-1].values
y = df.iloc[:,13].values

#df.head()
#print(X)

print(X.shape)

print(X)

(5684, 11)
[['CALHOUN-ISLE' 343.0 0.0 ... 0.0 'Wood Frame' 91500]
 ['CALHOUN-ISLE' 343.0 0.0 ... 0.0 'Wood Frame' 93000]
 ['CENTRAL' 344.0 0.0 ... 0.0 'Wood Frame' 80000]
 ...
 ['SOUTHWEST' 6565.0 3171.0 ... 2.0 'Concrete' 1400000]
 ['CALHOUN-ISLE' 6857.0 2733.0 ... 5.0 'Wood Frame' 2319500]
 ['CALHOUN-ISLE' 6972.0 3399.0 ... 2.0 'Wood Frame' 2992500]]


C:\Users\16123\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


## 4) Dealing with missing variables (part 2)
 ### Impute the numeric columns.

Note: leaving out the below ground square footage because we replcaed blanks with 0 already, so we won't want to impute on this column.

In [4]:
#From the scikit.impute library we first import the SimpleImputer class
#from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Imputer
#Next we define an object of the SimpleImputer class by looking at the docstring (use Shift+Tab)
imputer = Imputer(missing_values=np.nan, strategy='mean') 
imputer.fit(X[:,[1,3,5,6,7,8,10]])
X[:,[1,3,5,6,7,8,10]]= imputer.transform(X[:,[1,3,5,6,7,8,10]])
print(X[0])

['CALHOUN-ISLE' 343.0 0.0 11634.0 1963.0 1.0 0.0 1.0 0.0 'Wood Frame'
 91500.0]


C:\Users\16123\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## 5) Dealing with categorical variables
Using OneHotCoding on attributes 'Community' & 'ConstructionType'

Display: test row to verify all categorical variables are removed

In [6]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
X[:, 0] = labelencoder.fit_transform(X[:, 0])
X[:, 9] = labelencoder.fit_transform(X[:, 9])
print(X[0])

[0 343.0 0.0 11634.0 1963.0 1.0 0.0 1.0 0.0 5 91500.0]


In [7]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0,9])
data = onehotencoder.fit_transform(X).toarray()
#print(data[0])
NewData = data[:,[0,1,2,3,4,5,6,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25]]

X_sig = NewData[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]
print(X_sig[0])

[1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 1.0000e+00 3.4300e+02 0.0000e+00 1.1634e+04
 1.9630e+03 1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 9.1500e+04]


C:\Users\16123\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\16123\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [8]:
# Start of elimination: 
import statsmodels.api as sm
X_sig = NewData[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              2845.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        10:05:21   Log-Likelihood:                         -73863.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5660   BIC:                                  1.479e+05
Df Model:                          24                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.845e+04    1.3e+04      1.415      0.157   -7110.452     4.4e+04
x2          6066.9591   1.25e+04      0.485      0.628   -1.85e+04    3.06e+04
x3          3.304e+04   1.35e+04      2.442      0.015    6517.294    5.96e+04
x4          1.345e+04    1.3e+04      1.037      0.300    -1.2e+04    3.89e+04
x5          1.053e+04    1.3e+04      0.810      0.418   -1.49e+04     3.6e+04
x6          1.152e+04   1.26e+04      0.913      0.361   -1.32e+04    3.63e+04
x7          1.701e+04   1.28e+04      1.329      0.184   -8087.090    4.21e+04
x8          1.154e+04   1.25e+04      0.920      0.357    -1.3e+04    3.61e+04
x9          2.803e+04   1.26e+04      2.217      0.027    3242.828    5.28e+04
x10         2.564e+04   1.39e+04      1.841      0.066   -1667.543     5.3e+04
x11         7.176e+04    3.2e+04      2.244      0.025    9073.886    1.34e+05
x12         2.544e+04   3.06e+04      0.832      0.405   -3.45e+04    8.54e+04
x13         3.729e+04   3.08e+04      1.210      0.226   -2.31e+04    9.77e+04
x14         2.378e+04   3.53e+04      0.674      0.501   -4.54e+04     9.3e+04
x15         2.954e+04   3.01e+04      0.980      0.327   -2.95e+04    8.86e+04
x16           36.5475      6.623      5.518      0.000      23.564      49.531
x17           -3.2635      5.836     -0.559      0.576     -14.704       8.177
x18           -0.1356      0.045     -2.997      0.003      -0.224      -0.047
x19          -16.2686     16.477     -0.987      0.324     -48.570      16.032
x20         1.326e+04   2531.783      5.239      0.000    8299.498    1.82e+04
x21        -2767.7582   2117.663     -1.307      0.191   -6919.189    1383.673
x22        -1.045e+04   4285.003     -2.439      0.015   -1.89e+04   -2050.145
x23        -6195.8019   2889.871     -2.144      0.032   -1.19e+04    -530.548
x24            0.8557      0.015     57.384      0.000       0.826       0.885
==============================================================================
Omnibus:                    11366.823   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75308166.498
Skew:                          15.764   Prob(JB):                         0.00
Kurtosis:                     566.015   Cond. No.                     1.75e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 6) Remove a column from each of the OneHotEncoded variables groups with P>0.5 to find statistically significant variables

In [9]:
#E1: removed position 2 = column 1
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]

#Start: X_sig = NewData[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              2969.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        10:59:04   Log-Likelihood:                         -73863.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5661   BIC:                                  1.479e+05
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.317e+04   7168.761      1.837      0.066    -881.922    2.72e+04
x2          2.794e+04   8496.202      3.289      0.001    1.13e+04    4.46e+04
x3          8047.2587   6626.453      1.214      0.225   -4943.128     2.1e+04
x4          5107.3226   6623.883      0.771      0.441   -7878.026    1.81e+04
x5          6064.9783   5684.178      1.067      0.286   -5078.188    1.72e+04
x6          1.158e+04   6204.650      1.867      0.062    -579.604    2.37e+04
x7          6181.5579   5923.126      1.044      0.297   -5430.038    1.78e+04
x8          2.263e+04   5986.041      3.781      0.000    1.09e+04    3.44e+04
x9          2.045e+04   8897.217      2.298      0.022    3004.809    3.79e+04
x10         7.237e+04    3.2e+04      2.265      0.024    9732.752    1.35e+05
x11         2.592e+04   3.06e+04      0.848      0.396    -3.4e+04    8.58e+04
x12         3.764e+04   3.08e+04      1.222      0.222   -2.28e+04     9.8e+04
x13         2.414e+04   3.53e+04      0.684      0.494    -4.5e+04    9.33e+04
x14         3.004e+04   3.01e+04      0.998      0.318    -2.9e+04    8.91e+04
x15           36.4327      6.618      5.505      0.000      23.458      49.407
x16           -2.9379      5.797     -0.507      0.612     -14.301       8.426
x17           -0.1377      0.045     -3.059      0.002      -0.226      -0.049
x18          -13.7004     15.601     -0.878      0.380     -44.284      16.883
x19         1.325e+04   2531.506      5.235      0.000    8288.810    1.82e+04
x20        -2767.1587   2117.520     -1.307      0.191   -6918.308    1383.991
x21        -1.053e+04   4281.934     -2.458      0.014   -1.89e+04   -2130.968
x22        -6206.4068   2889.593     -2.148      0.032   -1.19e+04    -541.698
x23            0.8556      0.015     57.386      0.000       0.826       0.885
==============================================================================
Omnibus:                    11367.293   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75322755.714
Skew:                          15.766   Prob(JB):                         0.00
Kurtosis:                     566.069   Cond. No.                     1.75e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
#E2: removed position 16 = column 16
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23]]

#E1: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              3104.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:01:24   Log-Likelihood:                         -73863.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5662   BIC:                                  1.479e+05
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.396e+04   6998.730      1.994      0.046     237.006    2.77e+04
x2          2.891e+04   8277.739      3.492      0.000    1.27e+04    4.51e+04
x3          8077.3675   6625.752      1.219      0.223   -4911.645    2.11e+04
x4          5036.7619   6621.985      0.761      0.447   -7944.866     1.8e+04
x5          6099.7073   5683.392      1.073      0.283   -5041.918    1.72e+04
x6          1.164e+04   6203.298      1.876      0.061    -522.064    2.38e+04
x7          6512.9457   5886.542      1.106      0.269   -5026.932    1.81e+04
x8          2.286e+04   5968.091      3.831      0.000    1.12e+04    3.46e+04
x9          2.118e+04   8776.928      2.414      0.016    3977.876    3.84e+04
x10         7.224e+04   3.19e+04      2.261      0.024    9608.210    1.35e+05
x11         2.576e+04   3.05e+04      0.843      0.399   -3.41e+04    8.56e+04
x12         3.734e+04   3.08e+04      1.212      0.225    -2.3e+04    9.77e+04
x13         2.432e+04   3.53e+04      0.689      0.491   -4.49e+04    9.35e+04
x14         2.891e+04      3e+04      0.963      0.336      -3e+04    8.78e+04
x15           35.8780      6.527      5.497      0.000      23.083      48.673
x16           -0.1333      0.044     -3.018      0.003      -0.220      -0.047
x17          -13.7107     15.600     -0.879      0.379     -44.292      16.871
x18         1.327e+04   2531.114      5.242      0.000    8306.719    1.82e+04
x19        -3051.5158   2041.710     -1.495      0.135   -7054.049     951.017
x20        -1.013e+04   4208.699     -2.406      0.016   -1.84e+04   -1875.616
x21        -6561.3408   2803.268     -2.341      0.019   -1.21e+04   -1065.861
x22            0.8548      0.015     57.650      0.000       0.826       0.884
==============================================================================
Omnibus:                    11367.493   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75311764.039
Skew:                          15.766   Prob(JB):                         0.00
Kurtosis:                     566.028   Cond. No.                     1.75e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
#E3: removed position 13 = column 13
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,14,15,17,18,19,20,21,22,23]]

#E2: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              3252.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:03:21   Log-Likelihood:                         -73863.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5663   BIC:                                  1.479e+05
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.389e+04   6997.809      1.986      0.047     175.844    2.76e+04
x2          2.929e+04   8258.794      3.547      0.000    1.31e+04    4.55e+04
x3          8143.7276   6624.745      1.229      0.219   -4843.310    2.11e+04
x4          5085.8938   6621.294      0.768      0.442   -7894.379    1.81e+04
x5          6080.2674   5683.059      1.070      0.285   -5060.704    1.72e+04
x6          1.165e+04   6202.994      1.878      0.060    -511.716    2.38e+04
x7          6600.3948   5884.902      1.122      0.262   -4936.266    1.81e+04
x8          2.283e+04   5967.630      3.826      0.000    1.11e+04    3.45e+04
x9          2.058e+04   8732.124      2.356      0.018    3458.056    3.77e+04
x10          5.47e+04   1.93e+04      2.832      0.005    1.68e+04    9.26e+04
x11         7967.8034   1.63e+04      0.488      0.626    -2.4e+04       4e+04
x12          1.93e+04   1.62e+04      1.189      0.234   -1.25e+04    5.11e+04
x13         1.133e+04   1.59e+04      0.715      0.475   -1.97e+04    4.24e+04
x14           35.8016      6.526      5.486      0.000      23.009      48.594
x15           -0.1349      0.044     -3.058      0.002      -0.221      -0.048
x16           -4.6960      8.500     -0.552      0.581     -21.359      11.967
x17         1.309e+04   2518.367      5.200      0.000    8157.658     1.8e+04
x18        -2993.7686   2039.895     -1.468      0.142   -6992.744    1005.207
x19        -9972.7529   4202.605     -2.373      0.018   -1.82e+04   -1734.038
x20        -6514.2320   2802.305     -2.325      0.020    -1.2e+04   -1020.641
x21            0.8553      0.015     57.749      0.000       0.826       0.884
==============================================================================
Omnibus:                    11367.833   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75332856.420
Skew:                          15.767   Prob(JB):                         0.00
Kurtosis:                     566.107   Cond. No.                     7.99e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.99e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
#E4: removed position 16 = column 18
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,14,15,17,19,20,21,22,23]]

#E3: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,14,15,17,18,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              3415.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:05:13   Log-Likelihood:                         -73864.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5664   BIC:                                  1.479e+05
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.328e+04   6909.343      1.922      0.055    -262.067    2.68e+04
x2          2.771e+04   7747.944      3.577      0.000    1.25e+04    4.29e+04
x3          7683.4287   6571.742      1.169      0.242   -5199.702    2.06e+04
x4          4731.1011   6589.674      0.718      0.473   -8187.183    1.76e+04
x5          5576.1868   5608.993      0.994      0.320   -5419.588    1.66e+04
x6          1.117e+04   6140.969      1.818      0.069    -872.077    2.32e+04
x7          6133.5089   5823.550      1.053      0.292   -5282.880    1.75e+04
x8          2.237e+04   5907.336      3.786      0.000    1.08e+04    3.39e+04
x9          1.927e+04   8407.082      2.293      0.022    2792.283    3.58e+04
x10          4.74e+04   1.41e+04      3.365      0.001    1.98e+04     7.5e+04
x11          412.6322   8920.298      0.046      0.963   -1.71e+04    1.79e+04
x12         1.195e+04   9300.323      1.285      0.199   -6278.504    3.02e+04
x13         3347.1324   6514.231      0.514      0.607   -9423.254    1.61e+04
x14           35.6399      6.519      5.467      0.000      22.861      48.419
x15           -0.1363      0.044     -3.095      0.002      -0.223      -0.050
x16         1.299e+04   2511.261      5.173      0.000    8068.282    1.79e+04
x17        -3086.1881   2032.900     -1.518      0.129   -7071.450     899.073
x18        -1.008e+04   4198.002     -2.401      0.016   -1.83e+04   -1848.612
x19        -6516.0211   2802.131     -2.325      0.020    -1.2e+04   -1022.771
x20            0.8558      0.015     57.924      0.000       0.827       0.885
==============================================================================
Omnibus:                    11369.701   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75379078.167
Skew:                          15.773   Prob(JB):                         0.00
Kurtosis:                     566.280   Cond. No.                     4.55e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.55e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
#E5: removed position 11 = column 11
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,12,14,15,17,19,20,21,22,23]]

#E4: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,11,12,14,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              3596.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:07:24   Log-Likelihood:                         -73864.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5665   BIC:                                  1.479e+05
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.338e+04   6569.417      2.037      0.042     503.251    2.63e+04
x2          2.788e+04   6815.204      4.091      0.000    1.45e+04    4.12e+04
x3          7755.7419   6382.539      1.215      0.224   -4756.479    2.03e+04
x4          4780.6879   6501.326      0.735      0.462   -7964.400    1.75e+04
x5          5657.3757   5326.845      1.062      0.288   -4785.280    1.61e+04
x6          1.124e+04   5926.645      1.897      0.058    -377.615    2.29e+04
x7          6207.0593   5601.792      1.108      0.268   -4774.597    1.72e+04
x8          2.245e+04   5621.789      3.993      0.000    1.14e+04    3.35e+04
x9          1.939e+04   7997.918      2.425      0.015    3714.147    3.51e+04
x10         4.719e+04   1.33e+04      3.542      0.000    2.11e+04    7.33e+04
x11         1.172e+04   7874.985      1.489      0.137   -3713.116    2.72e+04
x12         3181.0874   5435.532      0.585      0.558   -7474.636    1.38e+04
x13           35.6882      6.434      5.547      0.000      23.075      48.301
x14           -0.1361      0.044     -3.100      0.002      -0.222      -0.050
x15           1.3e+04   2508.616      5.181      0.000    8078.569    1.79e+04
x16        -3076.6460   2022.228     -1.521      0.128   -7040.988     887.696
x17        -1.005e+04   4152.696     -2.420      0.016   -1.82e+04   -1909.092
x18        -6534.0410   2774.676     -2.355      0.019    -1.2e+04   -1094.613
x19            0.8557      0.015     58.804      0.000       0.827       0.884
==============================================================================
Omnibus:                    11369.490   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75370244.385
Skew:                          15.773   Prob(JB):                         0.00
Kurtosis:                     566.246   Cond. No.                     3.63e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.63e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
#E6: removed position 12 = column 14
X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,12,15,17,19,20,21,22,23]]

#E5: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,12,14,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              3796.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:09:03   Log-Likelihood:                         -73864.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5666   BIC:                                  1.479e+05
Df Model:                          18                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.426e+04   6393.987      2.231      0.026    1728.804    2.68e+04
x2          2.778e+04   6812.441      4.077      0.000    1.44e+04    4.11e+04
x3          8744.7719   6154.385      1.421      0.155   -3320.178    2.08e+04
x4          5242.3687   6452.913      0.812      0.417   -7407.810    1.79e+04
x5          6753.3604   4986.516      1.354      0.176   -3022.120    1.65e+04
x6          1.211e+04   5739.195      2.109      0.035     854.536    2.34e+04
x7          7092.6845   5393.209      1.315      0.189   -3480.070    1.77e+04
x8          2.333e+04   5416.650      4.307      0.000    1.27e+04    3.39e+04
x9           1.95e+04   7995.451      2.439      0.015    3823.725    3.52e+04
x10         4.561e+04    1.3e+04      3.497      0.000       2e+04    7.12e+04
x11         1.033e+04   7506.216      1.376      0.169   -4383.201     2.5e+04
x12           35.7925      6.431      5.565      0.000      23.185      48.400
x13           -0.1338      0.044     -3.059      0.002      -0.220      -0.048
x14         1.307e+04   2505.573      5.215      0.000    8155.083     1.8e+04
x15        -2662.8344   1894.473     -1.406      0.160   -6376.726    1051.058
x16        -9193.2554   3885.902     -2.366      0.018   -1.68e+04   -1575.400
x17        -6566.9951   2773.944     -2.367      0.018    -1.2e+04   -1129.003
x18            0.8543      0.014     59.566      0.000       0.826       0.882
==============================================================================
Omnibus:                    11367.222   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75293978.924
Skew:                          15.765   Prob(JB):                         0.00
Kurtosis:                     565.961   Cond. No.                     3.46e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.46e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
#E7: removed position 4 = column 4
X_sig = NewData[:,[0,2,3,5,6,7,8,9,10,12,15,17,19,20,21,22,23]]

#E6: X_sig = NewData[:,[0,2,3,4,5,6,7,8,9,10,12,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              4020.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:10:34   Log-Likelihood:                         -73864.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5667   BIC:                                  1.479e+05
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.316e+04   6247.966      2.106      0.035     911.972    2.54e+04
x2          2.685e+04   6715.484      3.998      0.000    1.37e+04       4e+04
x3          7527.4650   5969.012      1.261      0.207   -4174.084    1.92e+04
x4          5625.7152   4789.293      1.175      0.240   -3763.131     1.5e+04
x5          1.085e+04   5525.596      1.963      0.050      13.571    2.17e+04
x6          5815.9043   5158.988      1.127      0.260   -4297.686    1.59e+04
x7          2.218e+04   5229.732      4.242      0.000    1.19e+04    3.24e+04
x8          1.837e+04   7873.948      2.333      0.020    2934.913    3.38e+04
x9          4.584e+04    1.3e+04      3.516      0.000    2.03e+04    7.14e+04
x10         1.051e+04   7502.921      1.400      0.161   -4202.339    2.52e+04
x11           36.3575      6.393      5.687      0.000      23.824      48.891
x12           -0.1332      0.044     -3.046      0.002      -0.219      -0.047
x13         1.315e+04   2503.507      5.252      0.000    8240.258    1.81e+04
x14        -2546.2413   1888.972     -1.348      0.178   -6249.350    1156.867
x15        -8743.9369   3846.227     -2.273      0.023   -1.63e+04   -1203.861
x16        -6634.2225   2772.626     -2.393      0.017   -1.21e+04   -1198.814
x17            0.8524      0.014     60.241      0.000       0.825       0.880
==============================================================================
Omnibus:                    11364.562   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75164889.459
Skew:                          15.757   Prob(JB):                         0.00
Kurtosis:                     565.478   Cond. No.                     3.44e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.44e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
#E8: removed position 6 = column 7
X_sig = NewData[:,[0,2,3,5,6,8,9,10,12,15,17,19,20,21,22,23]]

#E7: X_sig = NewData[:,[0,2,3,5,6,7,8,9,10,12,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              4270.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:11:52   Log-Likelihood:                         -73865.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5668   BIC:                                  1.479e+05
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.143e+04   6056.958      1.887      0.059    -442.519    2.33e+04
x2          2.523e+04   6561.119      3.846      0.000    1.24e+04    3.81e+04
x3          5908.2715   5793.768      1.020      0.308   -5449.730    1.73e+04
x4          4168.3140   4611.621      0.904      0.366   -4872.227    1.32e+04
x5          9299.0780   5352.664      1.737      0.082   -1194.192    1.98e+04
x6          2.039e+04   4982.916      4.093      0.000    1.06e+04    3.02e+04
x7          1.665e+04   7725.417      2.156      0.031    1509.043    3.18e+04
x8          4.628e+04    1.3e+04      3.551      0.000    2.07e+04    7.18e+04
x9          1.052e+04   7503.092      1.402      0.161   -4190.380    2.52e+04
x10           36.0011      6.386      5.638      0.000      23.483      48.519
x11           -0.1290      0.044     -2.960      0.003      -0.214      -0.044
x12         1.309e+04   2503.017      5.229      0.000    8182.082     1.8e+04
x13        -2355.2484   1881.404     -1.252      0.211   -6043.521    1333.024
x14        -7915.7020   3775.497     -2.097      0.036   -1.53e+04    -514.283
x15        -6893.4468   2763.141     -2.495      0.013   -1.23e+04   -1476.634
x16            0.8541      0.014     60.716      0.000       0.827       0.882
==============================================================================
Omnibus:                    11361.556   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75098419.014
Skew:                          15.748   Prob(JB):                         0.00
Kurtosis:                     565.230   Cond. No.                     3.43e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
#E9: removed position 4 = column 5
X_sig = NewData[:,[0,2,3,6,8,9,10,12,15,17,19,20,21,22,23]]

#E8: X_sig = NewData[:,[0,2,3,5,6,8,9,10,12,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              4555.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:14:16   Log-Likelihood:                         -73865.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5669   BIC:                                  1.479e+05
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.007e+04   5867.629      1.717      0.086   -1429.144    2.16e+04
x2          2.423e+04   6466.674      3.747      0.000    1.16e+04    3.69e+04
x3          4753.6522   5651.100      0.841      0.400   -6324.665    1.58e+04
x4          8191.9998   5210.563      1.572      0.116   -2022.696    1.84e+04
x5          1.874e+04   4633.733      4.044      0.000    9653.661    2.78e+04
x6          1.548e+04   7614.958      2.033      0.042     549.647    3.04e+04
x7          4.641e+04    1.3e+04      3.561      0.000    2.09e+04     7.2e+04
x8          1.061e+04   7502.312      1.414      0.157   -4098.998    2.53e+04
x9            34.8972      6.268      5.568      0.000      22.610      47.184
x10           -0.1280      0.044     -2.938      0.003      -0.213      -0.043
x11         1.318e+04   2501.117      5.268      0.000    8273.013    1.81e+04
x12        -2061.1385   1853.023     -1.112      0.266   -5693.772    1571.495
x13        -7406.1008   3733.103     -1.984      0.047   -1.47e+04     -87.790
x14        -6585.9386   2742.072     -2.402      0.016    -1.2e+04   -1210.428
x15            0.8567      0.014     62.236      0.000       0.830       0.884
==============================================================================
Omnibus:                    11360.070   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75157962.486
Skew:                          15.742   Prob(JB):                         0.00
Kurtosis:                     565.454   Cond. No.                     3.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
#E10: removed position 3 = column 3
X_sig = NewData[:,[0,2,6,8,9,10,12,15,17,19,20,21,22,23]]

#E9: X_sig = NewData[:,[0,2,3,6,8,9,10,12,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              4881.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:16:47   Log-Likelihood:                         -73865.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5670   BIC:                                  1.479e+05
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          9325.0462   5799.599      1.608      0.108   -2044.386    2.07e+04
x2           2.36e+04   6422.955      3.674      0.000     1.1e+04    3.62e+04
x3          7566.6606   5157.126      1.467      0.142   -2543.279    1.77e+04
x4          1.792e+04   4529.570      3.955      0.000    9036.495    2.68e+04
x5          1.476e+04   7566.489      1.950      0.051     -75.462    2.96e+04
x6          4.643e+04    1.3e+04      3.563      0.000    2.09e+04     7.2e+04
x7          1.062e+04   7502.101      1.416      0.157   -4084.936    2.53e+04
x8            34.3468      6.233      5.510      0.000      22.127      46.566
x9            -0.1282      0.044     -2.943      0.003      -0.214      -0.043
x10         1.319e+04   2500.953      5.276      0.000    8292.087    1.81e+04
x11        -2011.3747   1852.031     -1.086      0.278   -5642.063    1619.313
x12        -6856.1614   3675.317     -1.865      0.062   -1.41e+04     348.866
x13        -6758.3828   2734.328     -2.472      0.013   -1.21e+04   -1398.055
x14            0.8584      0.014     63.021      0.000       0.832       0.885
==============================================================================
Omnibus:                    11358.899   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75144695.561
Skew:                          15.738   Prob(JB):                         0.00
Kurtosis:                     565.404   Cond. No.                     3.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
#E11: removed position 11 = column 20
X_sig = NewData[:,[0,2,6,8,9,10,12,15,17,19,21,22,23]]

#E10: X_sig = NewData[:,[0,2,6,8,9,10,12,15,17,19,20,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              5256.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:18:17   Log-Likelihood:                         -73866.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5671   BIC:                                  1.478e+05
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.011e+04   5754.487      1.757      0.079   -1171.087    2.14e+04
x2          2.453e+04   6365.439      3.854      0.000    1.21e+04     3.7e+04
x3          7041.0726   5134.450      1.371      0.170   -3024.413    1.71e+04
x4          1.754e+04   4516.596      3.884      0.000    8688.843    2.64e+04
x5          1.506e+04   7561.605      1.991      0.047     232.928    2.99e+04
x6          4.705e+04    1.3e+04      3.613      0.000    2.15e+04    7.26e+04
x7          1.076e+04   7501.187      1.434      0.152   -3947.964    2.55e+04
x8            31.4215      5.621      5.590      0.000      20.401      42.442
x9            -0.1239      0.043     -2.857      0.004      -0.209      -0.039
x10          1.25e+04   2417.702      5.170      0.000    7760.171    1.72e+04
x11        -7976.7370   3527.568     -2.261      0.024   -1.49e+04   -1061.355
x12        -6728.6587   2734.234     -2.461      0.014   -1.21e+04   -1368.514
x13            0.8620      0.013     65.309      0.000       0.836       0.888
==============================================================================
Omnibus:                    11372.388   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75588712.574
Skew:                          15.781   Prob(JB):                         0.00
Kurtosis:                     567.064   Cond. No.                     3.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
#E12: removed position 3 = column 6
X_sig = NewData[:,[0,2,8,9,10,12,15,17,19,21,22,23]]

#E11: X_sig = NewData[:,[0,2,6,8,9,10,12,15,17,19,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              5693.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:19:34   Log-Likelihood:                         -73867.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5672   BIC:                                  1.478e+05
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          9038.6711   5701.663      1.585      0.113   -2138.767    2.02e+04
x2          2.353e+04   6324.214      3.721      0.000    1.11e+04    3.59e+04
x3           1.65e+04   4452.520      3.706      0.000    7772.080    2.52e+04
x4          1.405e+04   7526.619      1.867      0.062    -703.094    2.88e+04
x5          4.738e+04    1.3e+04      3.640      0.000    2.19e+04    7.29e+04
x6          1.078e+04   7501.753      1.437      0.151   -3927.319    2.55e+04
x7            31.4335      5.622      5.591      0.000      20.413      42.454
x8            -0.1224      0.043     -2.822      0.005      -0.207      -0.037
x9          1.262e+04   2416.302      5.223      0.000    7883.074    1.74e+04
x10        -7572.8156   3515.522     -2.154      0.031   -1.45e+04    -681.048
x11        -6900.7778   2731.564     -2.526      0.012   -1.23e+04   -1545.868
x12            0.8627      0.013     65.392      0.000       0.837       0.889
==============================================================================
Omnibus:                    11383.037   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         76188774.034
Skew:                          15.813   Prob(JB):                         0.00
Kurtosis:                     569.302   Cond. No.                     3.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
#E13: removed position 6 = column 12
X_sig = NewData[:,[0,2,8,9,10,15,17,19,21,22,23]]

#E12: X_sig = NewData[:,[0,2,8,9,10,12,15,17,19,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              6209.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:22:25   Log-Likelihood:                         -73868.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5673   BIC:                                  1.478e+05
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          9241.4331   5700.451      1.621      0.105   -1933.630    2.04e+04
x2           2.76e+04   5657.500      4.878      0.000    1.65e+04    3.87e+04
x3          1.647e+04   4452.890      3.699      0.000    7741.462    2.52e+04
x4          1.567e+04   7442.634      2.105      0.035    1079.274    3.03e+04
x5           4.41e+04   1.28e+04      3.440      0.001     1.9e+04    6.92e+04
x6            31.7851      5.617      5.659      0.000      20.774      42.797
x7            -0.1149      0.043     -2.669      0.008      -0.199      -0.031
x8           1.24e+04   2411.663      5.142      0.000    7671.955    1.71e+04
x9         -7872.7723   3509.648     -2.243      0.025   -1.48e+04    -992.521
x10        -7187.7311   2724.509     -2.638      0.008   -1.25e+04   -1846.652
x11            0.8641      0.013     65.680      0.000       0.838       0.890
==============================================================================
Omnibus:                    11376.941   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75971399.139
Skew:                          15.794   Prob(JB):                         0.00
Kurtosis:                     568.493   Cond. No.                     3.31e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
#E14: removed position 1 = column 0
X_sig = NewData[:,[2,8,9,10,15,17,19,21,22,23]]

#E13: X_sig = NewData[:,[0,2,8,9,10,15,17,19,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              6828.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:24:35   Log-Likelihood:                         -73869.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5674   BIC:                                  1.478e+05
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2.541e+04   5494.658      4.624      0.000    1.46e+04    3.62e+04
x2           1.46e+04   4301.032      3.394      0.001    6166.259     2.3e+04
x3          1.395e+04   7367.464      1.893      0.058    -495.821    2.84e+04
x4          4.498e+04   1.28e+04      3.512      0.000    1.99e+04    7.01e+04
x5            31.2600      5.608      5.574      0.000      20.265      42.255
x6            -0.1045      0.043     -2.454      0.014      -0.188      -0.021
x7          1.234e+04   2411.722      5.116      0.000    7611.516    1.71e+04
x8         -7674.9854   3508.030     -2.188      0.029   -1.46e+04    -797.906
x9         -7139.7810   2724.740     -2.620      0.009   -1.25e+04   -1798.250
x10            0.8693      0.013     68.145      0.000       0.844       0.894
==============================================================================
Omnibus:                    11369.099   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75702568.934
Skew:                          15.769   Prob(JB):                         0.00
Kurtosis:                     567.491   Cond. No.                     3.31e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
#E15: removed position 3 = column 9
X_sig = NewData[:,[2,8,10,15,17,19,21,22,23]]

#E14: X_sig = NewData[:,[2,8,9,10,15,17,19,21,22,23]]
obj_OLS = sm.OLS(endog = y, exog = X_sig).fit() 
obj_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.923
Model:                            OLS   Adj. R-squared (uncentered):              0.923
Method:                 Least Squares   F-statistic:                              7583.
Date:                Wed, 06 May 2020   Prob (F-statistic):                        0.00
Time:                        11:25:52   Log-Likelihood:                         -73871.
No. Observations:                5684   AIC:                                  1.478e+05
Df Residuals:                    5675   BIC:                                  1.478e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          2.352e+04   5404.982      4.352      0.000    1.29e+04    3.41e+04
x2          1.364e+04   4272.437      3.194      0.001    5269.236     2.2e+04
x3          4.489e+04   1.28e+04      3.504      0.000    1.98e+04       7e+04
x4            30.5626      5.598      5.460      0.000      19.589      41.536
x5            -0.0885      0.042     -2.121      0.034      -0.170      -0.007
x6          1.232e+04   2412.254      5.108      0.000    7593.787    1.71e+04
x7         -7150.2822   3497.859     -2.044      0.041    -1.4e+04    -293.141
x8         -7406.8578   2721.704     -2.721      0.007   -1.27e+04   -2071.278
x9             0.8723      0.013     68.878      0.000       0.847       0.897
==============================================================================
Omnibus:                    11366.505   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         75494704.986
Skew:                          15.762   Prob(JB):                         0.00
Kurtosis:                     566.714   Cond. No.                     3.31e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.31e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
#End of Elimination, final column:
X_sig = NewData[:,[2,8,10,15,17,19,21,22,23]]

### Statistically significant variables
2: community-Central

8: community-Southwest

10: constructiontype-brick

15: abovegroundarea

17: parcel_area_sqft

19: bathrooms

21: num_stories

22: fireplaces

23: assessed_price